In [1]:
import torch
from tqdm import tqdm

# Image-related utilities
from torchvision.io import decode_image, read_image
from torchvision.transforms import ToTensor
from torchvision import transforms
from PIL import Image

# Import models
from torchvision.models import vgg19, VGG19_Weights

# Dataset
from torchvision.datasets import Imagenette, ImageFolder

# Plotting utility
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Read imagenette data into data loader
imagenette_data = ImageFolder(root='/home/yi/Downloads/imagenette2/val', transform=ToTensor())
data_loader = torch.utils.data.DataLoader(imagenette_data, batch_size=1, shuffle=False)

In [3]:
# Map number to class name
label_to_class = {
    0: "English springer",
    1: "French horn",
    2: "cassette player",
    3: "chain saw",
    4: "church",
    5: "garbage truck",
    6: "gas pump",
    7: "golf ball",
    8: "parachute",
    9: "tench"
}

label_to_id = {
    0: 217,
    1: 566,
    2: 482, 
    3: 491,
    4: 497,
    5: 569,
    6: 571, 
    7: 574,
    8: 701, 
    9: 0
}

In [4]:
weights = VGG19_Weights.DEFAULT
model = vgg19(weights=weights).to(device)
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [6]:
img_path = []
true_label = []
predicted_label = []
confidence = []
true_confidence = []
correctness = []

image_paths = [data_loader.dataset.samples[i][0] for i in range(len(data_loader.dataset))]

with torch.no_grad():
    for idx, (img, label) in enumerate(tqdm(data_loader, desc="Evaluating", unit="images")):
        # Retrieve image
        img = img.to(device)
        label = label.to(device)
        #Process image
        preprocess = weights.transforms()
        batch = preprocess(img)
        # Prediction
        prediction = model(batch).squeeze(0).softmax(0)
        class_id = prediction.argmax().item()
        score = prediction[class_id].item()
        true_score = prediction[label_to_id[label.item()]].item()
        # Correct?
        category_name = weights.meta["categories"][class_id]
        correct = 'y'
        if(category_name != label_to_class[label.item()]):
            correct = 'n'
        # Log
        img_path.append(image_paths[idx])
        true_label.append(label_to_class[label.item()])
        predicted_label.append(category_name)
        confidence.append(score)
        true_confidence.append(true_score)
        correctness.append(correct)

# Dictionary of logs
dict = {'image': img_path, 'true label': true_label, 'predicted label': predicted_label,
        'confidence score': confidence, 'true confidence score': true_confidence, 'correct': correctness}
df = pd.DataFrame(dict)
df.to_csv('vgg19_results.csv')

Evaluating: 100%|███████████████████████| 3925/3925 [00:58<00:00, 67.13images/s]
